<a href="https://colab.research.google.com/github/maochaokuo/pyKerasGA1/blob/master/twfutures1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tw futures tendency prediction by keras
## place to imort


In [1]:
import pandas as pd
from collections import deque
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint, ModelCheckpoint
import time
from sklearn import preprocessing
import datetime

print(datetime.datetime.now())

2019-07-27 12:47:44.411533


## orignal data to process (predict the tendency)

In [2]:
url='https://raw.githubusercontent.com/maochaokuo/tensorflow1/master/sharedfiles/aOHLCday.csv'
df = pd.read_csv(url, names=['dealdate','dealtime','open','high', 'low', 'close', 'volume'])  # read in specific file

import io
cols=["dealdate","dealtime","open","high", "low", "close", "volume"]
#df = pd.read_csv(url, names=cols, dtype=dict(zip(cols,[str, str, float, float, float, float, int])) ) 
#df.info()

#df['dealdate']=df['dealdate'].astype('|S')

#df.dealdate = df.dealdate.astype(str)
#df.dealtime = df.dealtime.astype(str)
#df.open=df.open.astype(float)
#df['dealtime']='0'+df['dealtime']
#df['dealtime']=df['dealtime'][-6:]
df=df.iloc[1:] #remove 1st row, the header

dropIndexes=df[ df['dealdate']>'20180703' ].index
df.drop(dropIndexes, inplace=True)

#df.groupby('dealdate')['open'].first()
#df.groupby('dealdate')['close'].last()
#df.groupby('dealdate')['low'].min()
#df.groupby('dealdate')['high'].max()
print(datetime.datetime.now()) # about 0.5 second
#df['dealdate'].dtype


2019-07-27 12:47:44.914029


In [3]:
#df1=df(['dealdate','dealtime','open','high','low','close','volume'], dtypes=[np.str,np.str,np.float64,np.float64,np.float64,np.float64,np.int64])
#df.dtypes
df.head()

,dealdate,dealtime,open,high,low,close,volume
1,20180629,084500,10541.00,10542.00,10529.00,10531.00,4426
2,20180629,084600,10531.00,10532.00,10526.00,10531.00,2142
3,20180629,084700,10530.00,10532.00,10530.00,10532.00,454
4,20180629,084800,10531.00,10538.00,10529.00,10536.00,1204
5,20180629,084900,10537.00,10537.00,10527.00,10527.00,1522


## df2 whole day summary (#1)

In [4]:
df2=pd.DataFrame( df.groupby('dealdate')['open'].first() )
df2=df2.join(df.groupby('dealdate')['high'].max())
df2=df2.join(df.groupby('dealdate')['low'].min())
df2=df2.join(df.groupby('dealdate')['close'].last())
df2=df2.join(df.groupby('dealdate')['dealdate'].last())
#df2['dealdate']=df2['name']
df2['hightime']='140000'
df2['lowtime']='140000'
print(datetime.datetime.now()) # about 0.2 second
df2.head()

2019-07-27 12:47:45.128732


,open,high,low,close,dealdate,hightime,lowtime
dealdate,,,,,,,
20180629,10541.00,10692.00,10526.00,10677.00,20180629,140000,140000
20180702,10680.00,10719.00,10609.00,10611.00,20180702,140000,140000
20180703,10651.00,10717.00,10544.00,10622.00,20180703,140000,140000
20180704,10598.00,10639.00,10584.00,10618.00,20180704,140000,140000
20180705,10625.00,10633.00,10488.00,10500.00,20180705,140000,140000


## df2 whole day summary (#2)

In [5]:
for index, row in df2.iterrows():
    df3=df[(df['dealdate']==row['dealdate']) & (df['high']==row['high'])] #first dealtime when high hit highest
    #row['hightime']=df3['dealtime'].iloc[0]
    df2.at[index,'hightime']=df3['dealtime'].iloc[0]
    df3=df[(df['dealdate']==row['dealdate']) & (df['low']==row['low'])] #first dealtime when low hit lowest
    df2.at[index,'lowtime']=df3['dealtime'].iloc[0]
    #row['hightime']=
    #df3.iloc[0]['hightime']  #df[['dealtime']][ (df['dealdate']==row['dealdate']) & (df['high']==row['high'])]
    
#df3= df[(df['dealdate']=='20180629') & (df['high']=='10692.00')] 
#df3.iloc[0]
print(datetime.datetime.now()) # about 5 seconds
df2.head()
#df2.tail()

2019-07-27 12:47:49.598314


,open,high,low,close,dealdate,hightime,lowtime
dealdate,,,,,,,
20180629,10541.00,10692.00,10526.00,10677.00,20180629,132900,084600
20180702,10680.00,10719.00,10609.00,10611.00,20180702,101500,134300
20180703,10651.00,10717.00,10544.00,10622.00,20180703,092500,112100
20180704,10598.00,10639.00,10584.00,10618.00,20180704,093800,111700
20180705,10625.00,10633.00,10488.00,10500.00,20180705,084500,131300


接下來算大趨勢

!!!>>>>> 目前這裡非常慢

In [6]:
#df.head(10)
#print(datetime.datetime.now())    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
daterow=None
curdate='20000101'
l=0
for index, row in df.iterrows():
    dealdate=str(row['dealdate'])
    dealtime=str(row['dealtime'])
    # daily summary
    if daterow is None or curdate<dealdate:
        daterow=df2[ df2['dealdate']==dealdate ].iloc[0] 
        datehightime=str(daterow['hightime'])
        datelowtime=str(daterow['lowtime'])
    else:
        curdate=dealdate
    if datehightime < datelowtime: # high earlier than low
        if dealtime < datehightime:
            df.at[index,'tendMajor']=1
        elif dealtime >= datehightime and dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        else:
            df.at[index,'tendMajor']=1
    else: # low earlier than high
        if dealtime < datelowtime:
            df.at[index,'tendMajor']=-1
        elif dealtime >= datelowtime and dealtime < datehightime:
            df.at[index,'tendMajor']=1
        else:
            df.at[index,'tendMajor']=-1
    l=l+1
print('l='+str(l))
#print(datetime.datetime.now())    # about 1:22 seconds, or 82 seconds
print(datetime.datetime.now().strftime("%m/%d/%Y, %H:%M:%S"))
#df.head(300)

07/27/2019, 12:47:49
l=77864
07/27/2019, 12:49:07


In [7]:
df.head(10)

,dealdate,dealtime,open,high,low,close,volume,tendMajor
1,20180629,084500,10541.00,10542.00,10529.00,10531.00,4426,-1.0
2,20180629,084600,10531.00,10532.00,10526.00,10531.00,2142,1.0
3,20180629,084700,10530.00,10532.00,10530.00,10532.00,454,1.0
4,20180629,084800,10531.00,10538.00,10529.00,10536.00,1204,1.0
5,20180629,084900,10537.00,10537.00,10527.00,10527.00,1522,1.0
6,20180629,085000,10527.00,10532.00,10526.00,10531.00,1308,1.0
7,20180629,085100,10531.00,10535.00,10529.00,10532.00,888,1.0
8,20180629,085200,10533.00,10534.00,10530.00,10530.00,506,1.0
9,20180629,085300,10530.00,10531.00,10528.00,10528.00,668,1.0
10,20180629,085400,10528.00,10533.00,10528.00,10533.00,608,1.0


接下來算小趨勢

In [16]:
curdate='20000101'
pre2index=-1
preindex=-1
pre2high=-1.0
pre2low=-1.0
prehigh=-1.0
prelow=-1.0
l=0
df['tendMinor']=0
for index, row in df.iterrows():
    if row['dealdate']=='dealdate':
        continue
    if row['dealdate'] > curdate: ## change date
        pre2index = -1
        preindex = -1 
        pre2high=-1.0
        pre2low=-1.0
        prehigh=-1.0
        prelow=-1.0
    curdate=row['dealdate']
    tendMinor = 0
    #print(pre2high)
    #print(pre2low)
    #print(prehigh)
    #print(prelow)
    if pre2high>0 and pre2low>0 and prehigh>0 and prelow>0:
        if prehigh > pre2high and prehigh >float(row['high']): #local high at prerow
            tendMinor = -1
        if prelow < pre2low and prelow <float( row['low']): #local low at prerow
            if tendMinor!=0:
                tendMinor = 0
            else:
                tendMinor = 1
    #print('1 preindex='+str(preindex))
    if preindex>0:
        #print('prerow not none')
        df.at[preindex,'tendMinor']=tendMinor
    #else:
    #  print('prerow none')
    # before end of for
    pre2index = preindex
    preindex = index
    #print('2 preindex='+str(preindex))
    pre2high=prehigh
    pre2low=prelow
    try:
        prehigh=float(row['high'])
        prelow=float(row['low'])
    except:
        print(row)
        break
    l=l+1
    #if l>300:
    #  break
print('l='+str(l))
print(datetime.datetime.now())  # about 14 seconds
df.head(10)

l=77864
2019-07-27 13:04:28.675515


,dealdate,dealtime,open,high,low,close,volume,tendMajor,tendMinor,testTend,baseprice,inventoryPL,inventoryPLmin,settlePL,settlePLmin,settlePLsum
1,20180629,084500,10541.00,10542.00,10529.00,10531.00,4426,-1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20180629,084600,10531.00,10532.00,10526.00,10531.00,2142,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20180629,084700,10530.00,10532.00,10530.00,10532.00,454,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20180629,084800,10531.00,10538.00,10529.00,10536.00,1204,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20180629,084900,10537.00,10537.00,10527.00,10527.00,1522,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,20180629,085000,10527.00,10532.00,10526.00,10531.00,1308,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20180629,085100,10531.00,10535.00,10529.00,10532.00,888,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,20180629,085200,10533.00,10534.00,10530.00,10530.00,506,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,20180629,085300,10530.00,10531.00,10528.00,10528.00,668,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,20180629,085400,10528.00,10533.00,10528.00,10533.00,608,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
#df.dtypes
#df

,dealdate,dealtime,open,high,low,close,volume,tendMajor,tendMinor,testTend,baseprice,inventoryPL,inventoryPLmin,settlePL,settlePLmin,settlePLsum
1,20180629,084500,10541.00,10542.00,10529.00,10531.00,4426,-1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20180629,084600,10531.00,10532.00,10526.00,10531.00,2142,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20180629,084700,10530.00,10532.00,10530.00,10532.00,454,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20180629,084800,10531.00,10538.00,10529.00,10536.00,1204,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20180629,084900,10537.00,10537.00,10527.00,10527.00,1522,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,20180629,085000,10527.00,10532.00,10526.00,10531.00,1308,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20180629,085100,10531.00,10535.00,10529.00,10532.00,888,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,20180629,085200,10533.00,10534.00,10530.00,10530.00,506,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,20180629,085300,10530.00,10531.00,10528.00,10528.00,668,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,20180629,085400,10528.00,10533.00,10528.00,10533.00,608,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
curdate='20010101'
pretend=0
baseprice=0
inventoryPLmin=0
settlePLmin=0
settlePLsum=0
for index, row in df.iterrows():
    if row['dealdate']>curdate:
        curdate=row['dealdate']
        pretend=0
        baseprice=0
    dealtime=row['dealtime']
    close=float(row['close'])
    if row['dealtime'] < '090500' or row['dealtime'] > '133500':
        curtend=0
    else:
        tendMajor=int(df.at[index, 'tendMajor'])
        if np.isnan( df.at[index, 'tendMinor'] ):
            print('wrong! '+curdate+' '+dealtime)
        tendMinor=int(df.at[index, 'tendMinor'])
        tendMinor=int(df.at[index, 'tendMinor'])
        if tendMinor!=0:
            curtend=tendMinor
        else:
            #curtend=pretend
            if pretend!=0:
                curtend=pretend
            else:
                curtend=tendMajor
    inventoryPL=0
    settlePL=0
    if curtend!=pretend:
        if pretend!=0 and baseprice==0:
            print('wrong! '+curdate+' '+dealtime)
            break
        settlePL=pretend*(close-baseprice)
        if settlePL<settlePLmin:
            settlePLmin = settlePL
        settlePLsum=settlePLsum+settlePL
        if curtend==0:
            baseprice=0
        else:
            baseprice=close
    else: #curtend==pretend
        inventoryPL=curtend*(close-baseprice)
        if inventoryPL<inventoryPLmin:
            inventoryPLmin = inventoryPL
    df.at[index, 'testTend']=curtend
    df.at[index, 'baseprice']=baseprice
    df.at[index, 'inventoryPL']=inventoryPL
    df.at[index, 'inventoryPLmin']=inventoryPLmin
    df.at[index, 'settlePL']=settlePL
    df.at[index, 'settlePLmin']=settlePLmin
    df.at[index, 'settlePLsum']=settlePLsum
    pretend=curtend
df.drop(["open","high","low","volume"], axis=1)
#df[ df['dealdate']=='20180702']    

,dealdate,dealtime,close,tendMajor,tendMinor,testTend,baseprice,inventoryPL,inventoryPLmin,settlePL,settlePLmin,settlePLsum
1,20180629,084500,10531.00,-1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,20180629,084600,10531.00,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,20180629,084700,10532.00,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,20180629,084800,10536.00,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,20180629,084900,10527.00,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,20180629,085000,10531.00,1.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,20180629,085100,10532.00,1.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,20180629,085200,10530.00,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,20180629,085300,10528.00,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,20180629,085400,10533.00,1.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
#df[ df['dealdate']=='20180702']